In [1]:
from utils import import_raw_data
from validation import (
    validate_ql_DeepAR, 
    validate_ql_intermittentGP, 
    validate_ql_baseline, 
    validate_ql_DLinear, 
    validate_smooth_ts,
    validate_local_fn
)
from reconciliation import temporal_recon_ts
from forecasters import gp
import numpy as np
import pandas as pd
import torch
#from statsmodels.tsa.holtwinters import ExponentialSmoothing
#from tweediegp.intermittent_gp import intermittentGP


/Users/stefano.damato/anaconda3/envs/VN2/lib/python3.11/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


## Validating local methods on smooth time series

In [ ]:
datasets = import_raw_data()

experiments = []

for likelihood in ["negbin", "gaussian"]:
    for kernel in [None, "periodic"]:
        for threshold in [3.5, 4., 4.5, 5.]:
            for h_recon in [3, 4]:

                torch.manual_seed(42)
                np.random.seed(42)

                exp_dict = {"likelihood":likelihood, "kernel":kernel, "h_recon":h_recon, "threshold":threshold}
                print(exp_dict)

                sales, in_stock, master = import_raw_data()
                in_stock = in_stock.iloc[:,:sales.shape[1]]
                colmeans = np.nanmean(np.where(in_stock, sales, np.nan), axis=0)
                in_stock.iloc[:,(colmeans > threshold)] = False
                    
                def negbin_gp(train_y, train_x, test_x):
                        return gp(train_y, train_x, test_x, likelihood="negbin")
                def gp_fn(train_y, train_x, test_x):
                        return gp(train_y, train_x, test_x, likelihood=likelihood, kernel=kernel)
                
                def recon_fn(train_y, observed, h):
                    if torch.all(train_y[observed] > 0.):
                        fn = gp_fn
                    else:
                        fn = negbin_gp
                    recon_samples, _ = temporal_recon_ts(fn, train_y, observed, h_recon)
                    return torch.tensor(recon_samples[:,:h])

                validation_results = validate_local_fn((sales, in_stock, master), recon_fn)

                for T in validation_results.keys():
                    experiments.append(
                        pd.Series(exp_dict | {
                            "T":T,
                            "QL simple":validation_results[T]["quantile_loss_simple"],
                            "QL cumulative":validation_results[T]["quantile_loss_cumulative"]
                        })
                    )

experiments = pd.DataFrame(experiments)

{'likelihood': 'negbin', 'kernel': None, 'h_recon': 3, 'threshold': 3.5}


100%|██████████| 599/599 [04:59<00:00,  2.00it/s]


{'likelihood': 'negbin', 'kernel': None, 'h_recon': 4, 'threshold': 3.5}


100%|██████████| 599/599 [04:46<00:00,  2.09it/s]


{'likelihood': 'negbin', 'kernel': None, 'h_recon': 3, 'threshold': 4.0}


...

100%|██████████| 599/599 [05:29<00:00,  1.82it/s]


{'likelihood': 'negbin', 'kernel': None, 'h_recon': 4, 'threshold': 5.0}


100%|██████████| 599/599 [05:08<00:00,  1.94it/s]


{'likelihood': 'negbin', 'kernel': 'periodic', 'h_recon': 3, 'threshold': 3.5}


 83%|████████▎ | 497/599 [04:16<00:48,  2.10it/s]

In [ ]:
experiments_grouped = experiments.groupby(exp_dict.keys(), dropna=False).agg(["mean", "std"])
experiments_grouped.sort_values(("QL simple", "mean"))

T           QL simple            \
                              mean       std      mean       std   
likelihood kernel threshold                                        
negbin     NaN    4.0        155.0  1.581139  0.968689  0.043263   
gaussian   NaN    3.5        155.0  1.581139  0.977666  0.021626   
                  4.0        155.0  1.581139  0.978613  0.031331   
negbin     NaN    3.5        155.0  1.581139  0.978798  0.050667   

                            QL cumulative            
                                     mean       std  
likelihood kernel threshold                          
negbin     NaN    4.0            1.891486  0.195400  
gaussian   NaN    3.5            1.941736  0.153445  
                  4.0            1.880134  0.186048  
negbin     NaN    3.5            2.028770  0.262390